Building recommendation systems using SVD++ and ItemKNN
---

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
file_path = r"C:\Users\vahav\OneDrive - HvA\Courses\System Development for Marketing\E-commerce\ecommerce_data.csv"
e_commerce = pd.read_csv(file_path)

# Set the display option to show all columns
pd.set_option('display.max_columns', None)

(e_commerce.head())

,Customer_ID,Age,Gender,Income_Level,Marital_Status,Education_Level,Occupation,Location,Purchase_Category,Purchase_Amount,Frequency_of_Purchase,Purchase_Channel,Brand_Loyalty,Product_Rating,Time_Spent_on_Product_Research(hours),Social_Media_Influence,Discount_Sensitivity,Return_Rate,Customer_Satisfaction,Engagement_with_Ads,Device_Used_for_Shopping,Payment_Method,Time_of_Purchase,Discount_Used,Customer_Loyalty_Program_Member,Purchase_Intent,Shipping_Preference,Time_to_Decision,Age_Group,Purchase_Amount_Group
0,37-611-6911,22,Female,Middle,Married,Bachelor's,Middle,Évry,Gardening & Outdoors,333.80,4,Mixed,5,5,2.0,NaN,Somewhat Sensitive,1,7,NaN,Tablet,Credit Card,3/1/2024,True,False,Need-based,No Preference,2,18–25,$251–350
1,29-392-9296,49,Male,High,Married,High School,High,Huocheng,Food & Beverages,222.22,11,In-Store,3,1,2.0,Medium,Not Sensitive,1,5,High,Tablet,PayPal,4/16/2024,True,False,Wants-based,Standard,6,46–50,$151–250
2,84-649-5117,24,Female,Middle,Single,Master's,High,Huzhen,Office Supplies,426.22,2,Mixed,5,5,0.3,Low,Not Sensitive,1,7,Low,Smartphone,Debit Card,3/15/2024,True,True,Impulsive,No Preference,3,18–25,$351–450
3,48-980-6078,29,Female,Middle,Single,Master's,Middle,Wiwilí,Home Appliances,101.31,6,Mixed,3,1,1.0,High,Somewhat Sensitive,0,1,NaN,Smartphone,Other,10/4/2024,True,True,Need-based,Express,10,26–30,$50–150
4,91-170-9072,33,Female,Middle,Widowed,High School,Middle,Nara,Furniture,211.70,6,Mixed,3,4,0.0,Medium,Not Sensitive,2,10,NaN,Smartphone,Debit Card,1/30/2024,False,False,Wants-based,No Preference,4,31–35,$151–250


---
---
Model 1: SVD++
---
SVD++ is an advanced version of the Singular Value Decomposition (SVD) algorithm, used in collaborative filtering for recommendation systems. It is particularly effective for predicting user-item interactions by learning latent factors from the data, with an enhancement that takes into account implicit feedback (user behaviors) in addition to explicit ratings (the actual item ratings). The Surprise library is a popular Python library designed for building and evaluating recommender systems, and it includes an implementation of SVD++.

SVD++ in Surprise builds upon the basic SVD algorithm by incorporating implicit feedback (such as views, clicks, or purchases) along with explicit feedback (ratings). This makes it particularly effective when you have sparse rating data, but can gather additional implicit feedback from user interactions. The model works by decomposing the user-item interaction matrix into three matrices:
- User matrix (U): A matrix that represents latent features associated with users.
- Item matrix (V): A matrix that represents latent features associated with items.
- Implicit feedback matrix: This matrix captures the implicit interactions of the users with the items (such as viewing, clicking, etc.).

For SVD++ in Surprise, the input data needs to be in the following format: A user-item interaction dataset: A dataset where each entry represents a user's rating of an item, where:
- UserID: Identifies the user
- ItemID: Identifies the item
- Rating: The rating the user gave to the item

Surprise offers the SVDpp class, which can be instantiated with parameters:
- n_factors: The number of latent factors (typically between 50 and 200).
- lr_all: The learning rate for the gradient descent optimization.
- reg_all: The regularization parameter to prevent overfitting.
- n_epochs: The number of iterations to train the model.


**NOTE:**
Because in this dataset there is record of only one purhcase for each user, we will have to think about recommending users to items rather than items to users. So for the user-item matrix, this means having in each row all the different items (purchase category) and on columns each user (customer).

The underlying logic: "“Which users are most likely to engage with (or be targeted for) a given product category?”"

- Standard setup: User = Customer_ID, Item = Purchase_Category, Rating = Product_Rating
- Flipped setup: User = Purchase_Category, Item = Customer_ID, Rating = Product_Rating

---
The objective of this model is to build a personalized recommender system using SVD++ from Surprise, and evaluate performance on an e-commerce dataset by leveraging both standard product ratings and a custom rating formula. The model outputs are then later evaluated using both individual user and group fairness metrics. The process is as follows:

1. Flipped User-Item Structure: Instead of the typical user → item setup, you flipped the structure:
    - User = Product Category
    - Item = Customer
    - This allowes to create recommendations of customers to product categories, which can tehn be switched to get the top 10 predictions for each user. This was done, because the dataset only has 1 user-item interaction for each customer (= 1 rating per customer).

2. Standard Model Training
    - A baseline SVD++ model is trained using just the raw product ratings (Product_Rating) as the prediction target.
    - Surprise's Trainset and testset structures were used for model fitting and evaluation.
    - The model evaluated happens using:
        - RMSE (Root Mean Squared Error)
        - MAE (Mean Absolute Error)

3. Hyperparameter Tuning (Grid Search)
    - Performing GridSearchCV over multiple parameter combinations (n_factors, n_epochs, lr_all, reg_all).
    - Selecting the best model based on lowest RMSE.

4. Custom Rating Design
    - Introducing a custom rating that combines:
        - Product rating (50%)
        - Customer satisfaction (30%)
        - Brand loyalty (20%)
    - This better reflects overall customer sentiment and product quality.

5. New SVD++ Model on Custom Ratings
    - Repeating the train-test split and model training using the custom rating.
    - Adjusting the rating scale to match the new composite score range.
    - Re-running GridSearchCV to find optimal hyperparameters for the custom rating model.
    - Builing and evaluating a final SVD++ model using the best hyperparameters.

6. Top-N Recommendation Generation
    - Generating top-10 recommended customers per category.
    - Flipping the normal logic back and generating top-10 recommended categories per customer.

---
Printing product category stats:

In [3]:
# Compute number of reviews and average rating per product category
category_stats = e_commerce.groupby('Purchase_Category')['Product_Rating'].agg(['count', 'mean']).reset_index()
category_stats.columns = ['Purchase_Category', 'Num_Reviews', 'Avg_Rating']

# Sort by most reviewed categories
category_stats.sort_values(by='Num_Reviews', ascending=False)

,Purchase_Category,Num_Reviews,Avg_Rating
6,Electronics,54,2.888889
21,Sports & Outdoors,51,2.882353
13,Home Appliances,50,2.940000
15,Jewelry & Accessories,50,2.760000
22,Toys & Games,47,3.468085
0,Animal Feed,44,3.000000
7,Food & Beverages,44,2.704545
9,Gardening & Outdoors,44,2.568182
20,Software & Apps,43,3.162791
19,Packages),43,3.046512


It seems that each category has enough reviews so we don't have to drop any of them. However, it's notable to say that the average reviews for each are pretty low. Nothing is above 4.

Imports the necessary classes from the Surprise library to build, train, and evaluate your SVD++ recommender model:
- SVDpp: An algorithm used in collaborative filtering for recommendation systems. It works by decomposing the user-item interaction matrix into latent factors (user and item features) and then optimizing them to predict ratings. SVD++ improves upon traditional SVD by also incorporating implicit feedback (e.g., clicks or views), making it more effective in sparse datasets.  The implicit feedback is automatically handled by the model.
- Dataset: A class in Surprise that provides methods to load data from various sources (e.g., pandas DataFrame, CSV files, etc.) and prepares it in a format that is compatible with Surprise's algorithms. It structures the data as a user-item interaction matrix.
- Reader: A utility class in Surprise that defines how the data should be interpreted when loaded. For example, it allows you to specify the rating scale (e.g., 1 to 5) and the structure of your data (e.g., columns for user ID, item ID, and rating). It's necessary to set up Surprise with the correct data format.
- Trainset: A class in Surprise that holds the training data used by the recommendation system algorithm. It is created after the dataset is split (usually with train_test_split) and provides the necessary data structure for training the model (e.g., SVD++).
- accuracy: A module in Surprise that provides evaluation metrics (such as RMSE, MAE) to assess model performance. These metrics compare the predicted ratings with the actual ratings in the test set, helping you gauge the model's effectiveness.

Others:
- train_test_split: utility function from Scikit-learn used to split your dataset into training and testing subsets.
- defaultdict: a special kind of dictionary that automatically initializes default values if a key does not exist.


In [4]:
from surprise import SVDpp, Dataset, Reader, accuracy
from sklearn.model_selection import train_test_split
from collections import defaultdict

Defining the User-Item Structure -  flipping the logic to treat categories as users and customers as items, so that the model recommends customers for each product category (because each customer only rates one category):

In [5]:
# Flip logic: Treat product categories as 'users' and customers as 'items'
ratings_df = e_commerce[['Purchase_Category', 'Customer_ID', 'Product_Rating']].copy()
ratings_df.columns = ['userID', 'itemID', 'rating']

Performing a stratified 80/20 train-test split for each category so that each category is fairly represented in both sets:

In [6]:
# Manually split by product category (userID)
train_df = pd.DataFrame(columns=ratings_df.columns)
test_df = pd.DataFrame(columns=ratings_df.columns)

for category in ratings_df['userID'].unique():
    category_ratings = ratings_df[ratings_df['userID'] == category]
    train, test = train_test_split(category_ratings, test_size=0.2, random_state=42)
    train_df = pd.concat([train_df, train])
    test_df = pd.concat([test_df, test])

Results will be that each category will be represented in both sets (no cold-start issue for categories)
- train_df containing 80% of each category's ratings
- test_df containing 20% of each category's ratings

Confirming that the train-test split stayed close to 80/20 for each category:

In [7]:
# Validate that each category has an ~80/20 split
full_counts = ratings_df['userID'].value_counts().sort_index()
train_counts = train_df['userID'].value_counts().sort_index()
test_counts = test_df['userID'].value_counts().sort_index()

verification_df = pd.DataFrame({
    'Total_Ratings': full_counts,
    'Train_Ratings': train_counts,
    'Test_Ratings': test_counts
})

verification_df['Train_%'] = (verification_df['Train_Ratings'] / verification_df['Total_Ratings'] * 100).round(1)
verification_df['Test_%'] = (verification_df['Test_Ratings'] / verification_df['Total_Ratings'] * 100).round(1)

display(verification_df)

issues = verification_df[(verification_df['Train_%'] < 75) | (verification_df['Train_%'] > 85)]
if not issues.empty:
    print("Categories with unexpected train/test split:")
    display(issues)
else:
    print("All categories have a valid 80/20 train/test split.")

,Total_Ratings,Train_Ratings,Test_Ratings,Train_%,Test_%
userID,,,,,
Animal Feed,44,35,9,79.5,20.5
Arts & Crafts,34,27,7,79.4,20.6
Baby Products,41,32,9,78.0,22.0
Beauty & Personal Care,34,27,7,79.4,20.6
Books,37,29,8,78.4,21.6
Clothing,30,24,6,80.0,20.0
Electronics,54,43,11,79.6,20.4
Food & Beverages,44,35,9,79.5,20.5
Furniture,41,32,9,78.0,22.0


All categories have a valid 80/20 train/test split.


Defining rating scale and format data:

In [8]:
# Define rating scale
reader = Reader(rating_scale=(1, 5))

# Build Surprise trainset object from train_df
trainset = Dataset.load_from_df(train_df[['userID', 'itemID', 'rating']], reader).build_full_trainset()

# Format test_df as Surprise testset (list of tuples)
testset = list(test_df[['userID', 'itemID', 'rating']].itertuples(index=False, name=None))

The code above converts the ratings_df into a Surprise-compatible format using the Reader object.

Moving to prepare train and test sets for Surprise:

In [9]:
# Convert training and testing DataFrames into Surprise formats
"""
The training set is converted using Surprise's 'build_full_trainset()', which transforms the DataFrame into a special Trainset object used for model fitting.
The test set is converted into a list of (userID, itemID, rating) tuples, which is the required input format for the model.test() method during evaluation."
"""

trainset = Dataset.load_from_df(train_df[['userID', 'itemID', 'rating']], reader).build_full_trainset()
testset = list(test_df[['userID', 'itemID', 'rating']].itertuples(index=False, name=None))

- trainset: Surprise’s internal object used to train models.
- testset: List of tuples in (userID, itemID, rating) format, needed to evaluate the model manually using .test().

Train and Predict with SVD++:

In [10]:
# Initialize the SVD++ model
svd_model = SVDpp()

# Train the model on the training set
svd_model.fit(trainset)

# Predict on the test set
svd_predictions = svd_model.test(testset)

Evaluating model performance:

In [11]:
# Calculate and print RMSE
svd_rmse = accuracy.rmse(svd_predictions)

# Calculate and print MAE
svd_mae = accuracy.mae(svd_predictions)

RMSE: 1.3942
MAE:  1.2176


It seems that the model is not performing so good. An RMSE of 1.39 and MAE of 1.21 are relatively high when the ratings are on a 1–5 scale, meaning the model predictions are off by more than a full rating point on average.

Performing a grid search to find the best hyperparameters for the SVD++ model using the product ratings and cross-validation:

In [12]:
from surprise.model_selection import GridSearchCV
"""GridSearchCV a utility in the Surprise library that performs hyperparameter tuning for collaborative filtering models (like SVD++, KNN, etc.) using cross-validation. 
It helps find the best combination of hyperparameters for the model by evaluating multiple combinations of parameters and selecting the one that gives the best performance."""

# Defineing parameter grid
param_grid = {
    'n_factors': [20, 50, 100],
    'n_epochs': [20, 40, 60],
    'lr_all': [0.003, 0.005, 0.01],
    'reg_all': [0.02, 0.05, 0.1]
}

# Running the grid search with 3-fold cross-validation
gs = GridSearchCV(SVDpp, param_grid, measures=['rmse'], cv=3, joblib_verbose=1)
gs.fit(Dataset.load_from_df(ratings_df, reader))

# Displaing the best RMSE and corresponding hyperparameters
print("Best RMSE:", gs.best_score['rmse'])
print("Best Parameters:", gs.best_params['rmse'])

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    2.5s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:   24.6s


Best RMSE: 1.4437777162122238
Best Parameters: {'n_factors': 100, 'n_epochs': 60, 'lr_all': 0.01, 'reg_all': 0.1}


The RMSE from the GridSearch is higher than with the baseline model. Defining custom rating that better reflects user preferences by blending product rating, customer satisfaction, and brand loyalty might help with the model performance. The weights reflect their relative importance:

In [13]:
#Creating a custom rating that incorporates product ratings, customer satisfaction, and brand loyalty, with the weights reflecting teh relative importance of each factor
e_commerce['custom_rating'] = (
    0.5 * e_commerce['Product_Rating'] +
    0.3 * e_commerce['Customer_Satisfaction'] +
    0.2 * e_commerce['Brand_Loyalty']
)

(e_commerce['custom_rating'].head())

0    5.6
1    2.6
2    5.6
3    1.4
4    5.6
Name: custom_rating, dtype: float64

Preparing Surprise Dataset for custom rating:

In [14]:
# Create flipped user-item dataset using custom ratings
new_ratings_df = e_commerce[['Purchase_Category', 'Customer_ID', 'custom_rating']].copy()
new_ratings_df.columns = ['userID', 'itemID', 'rating']  # flipped

# Define the rating scale for custom scores
new_reader = Reader(rating_scale=(1, 6.5))
new_data = Dataset.load_from_df(new_ratings_df, new_reader)

In [15]:
new_ratings_df.head()

,userID,itemID,rating
0,Gardening & Outdoors,37-611-6911,5.6
1,Food & Beverages,29-392-9296,2.6
2,Office Supplies,84-649-5117,5.6
3,Home Appliances,48-980-6078,1.4
4,Furniture,91-170-9072,5.6


Performing an 80/20 stratisfied split across all product categories with using the custom rating to ensure balanced evaluation in training and testing:

In [16]:
# Manually split by category to maintain class balance
new_train_df = pd.DataFrame(columns=new_ratings_df.columns)
new_test_df = pd.DataFrame(columns=new_ratings_df.columns)

for new_category in new_ratings_df['userID'].unique():
    new_category_ratings = new_ratings_df[new_ratings_df['userID'] == new_category]
    train, test = train_test_split(new_category_ratings, test_size=0.2, random_state=42)
    new_train_df = pd.concat([new_train_df, train])
    new_test_df = pd.concat([new_test_df, test])

C:\Users\vahav\AppData\Local\Temp\ipykernel_10744\1352253803.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_train_df = pd.concat([new_train_df, train])
C:\Users\vahav\AppData\Local\Temp\ipykernel_10744\1352253803.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_test_df = pd.concat([new_test_df, test])


Formatting train/test split for Surprise:

In [17]:
# Build Surprise-compatible trainset and testset
new_trainset = Dataset.load_from_df(new_train_df[['userID', 'itemID', 'rating']], new_reader).build_full_trainset()
new_testset = list(new_test_df[['userID', 'itemID', 'rating']].itertuples(index=False, name=None))

Training an initial SVD++ model on the custom ratings, and evaluating its performance using RMSE and MAE:

In [18]:
# Train SVD++ model
new_svd_model = SVDpp()
new_svd_model.fit(new_trainset)

# Predict and evaluate on test set
new_svd_predictions = new_svd_model.test(new_testset)
new_svd_rmse = accuracy.rmse(new_svd_predictions)
new_svd_mae = accuracy.mae(new_svd_predictions)

RMSE: 1.1346
MAE:  0.9026


Optimizing and retraining the SVD++ model using the best hyperparameters found with custom ratings:

In [19]:
# Run hyperparameter tuning on custom rating dataset
gs = GridSearchCV(SVDpp, param_grid, measures=['rmse'], cv=3, joblib_verbose=1)
gs.fit(new_data)

# Train final model using best params
best_params = gs.best_params['rmse']
final_svdpp_model = SVDpp(**best_params, random_state=42)
final_svdpp_model.fit(new_trainset)
final_svdpp_predictions = final_svdpp_model.test(new_testset)
final_svd_rmse = accuracy.rmse(final_svdpp_predictions)
final_svd_mae = accuracy.mae(final_svdpp_predictions)

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    2.5s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:   24.5s


RMSE: 1.1351
MAE:  0.9069


Looping over all combinations of categories and customers, storing and ranking the top 10 customers per product category based on predicted satisfaction:

In [20]:
"""
This block predicts the top 10 customers per product category
based on the final tuned SVD++ model.
"""
# Get all unique product categories and customers
categories = new_ratings_df['userID'].unique()    # product categories
customers = new_ratings_df['itemID'].unique()     # customer IDs

# Predict for all (category, customer) pairs
top_10_customers_per_category = defaultdict(list)

for category in categories:
    for customer in customers:
        prediction = final_svdpp_model.predict(uid=category, iid=customer)
        top_10_customers_per_category[category].append((customer, prediction.est))

# Keep top-10 per category
for category in top_10_customers_per_category:
    top_10_customers_per_category[category] = sorted(
        top_10_customers_per_category[category],
        key=lambda x: x[1], reverse=True
    )[:10]

# Print results for 3 categories
print("Top 10 customer recommendations for 3 sample product categories:\n")

for i, (category, top_customers) in enumerate(top_10_customers_per_category.items()):
    if i >= 3:
        break
    print(f"Category: {category}")
    for idx, (cust, rating) in enumerate(top_customers, 1):
        print(f"{idx}. Customer ID: {cust} — Predicted Rating: {rating:.2f}")
    print("=" * 60)

Top 10 customer recommendations for 3 sample product categories:

Category: Gardening & Outdoors
1. Customer ID: 38-912-1128 — Predicted Rating: 5.96
2. Customer ID: 37-611-6911 — Predicted Rating: 5.34
3. Customer ID: 86-068-2133 — Predicted Rating: 5.07
4. Customer ID: 89-730-1134 — Predicted Rating: 5.07
5. Customer ID: 16-583-2801 — Predicted Rating: 4.97
6. Customer ID: 64-706-7841 — Predicted Rating: 4.86
7. Customer ID: 99-477-8256 — Predicted Rating: 4.77
8. Customer ID: 64-180-2404 — Predicted Rating: 4.71
9. Customer ID: 17-328-4678 — Predicted Rating: 4.31
10. Customer ID: 20-939-3906 — Predicted Rating: 4.27
Category: Food & Beverages
1. Customer ID: 35-879-0014 — Predicted Rating: 5.18
2. Customer ID: 22-352-0576 — Predicted Rating: 5.14
3. Customer ID: 11-719-8800 — Predicted Rating: 5.09
4. Customer ID: 76-382-7162 — Predicted Rating: 4.96
5. Customer ID: 63-625-5757 — Predicted Rating: 4.57
6. Customer ID: 04-213-4617 — Predicted Rating: 4.46
7. Customer ID: 23-949-1954

Flipping the logic back and building personalized recommendations for each customer by ranking product categories based on predicted satisfaction:

In [21]:
# Recommend top categories per customer
SVDpp_top_10_categories_per_customer = defaultdict(list)

for customer in customers:
    for category in categories:
        prediction = final_svdpp_model.predict(uid=category, iid=customer)
        SVDpp_top_10_categories_per_customer[customer].append((category, prediction.est))

# Keep top 10 categories per customer
for customer in SVDpp_top_10_categories_per_customer:
    SVDpp_top_10_categories_per_customer[customer] = sorted(
        SVDpp_top_10_categories_per_customer[customer],
        key=lambda x: x[1], reverse=True
    )[:10]


print("Top 10 recommended product categories for 5 sample customers:\n")

for i, (customer, recs) in enumerate(SVDpp_top_10_categories_per_customer.items()):
    if i >= 5:
        break
    print("=" * 60)
    print(f"Customer ID: {customer}")
    print("Top 10 Recommended Product Categories:")
    print("-" * 60)
    for idx, (category, rating) in enumerate(recs, 1):
        print(f"{idx:>2}. {category:<35} | Predicted Rating: {rating:.2f}")
    print("=" * 60 + "\n")

Top 10 recommended product categories for 5 sample customers:

Customer ID: 37-611-6911
Top 10 Recommended Product Categories:
------------------------------------------------------------
 1. Gardening & Outdoors                | Predicted Rating: 5.34
 2. Baby Products                       | Predicted Rating: 4.93
 3. Clothing                            | Predicted Rating: 4.48
 4. Travel & Leisure (Flights           | Predicted Rating: 4.41
 5. Hotels                              | Predicted Rating: 4.31
 6. Packages)                           | Predicted Rating: 4.25
 7. Beauty & Personal Care              | Predicted Rating: 4.25
 8. Luxury Goods                        | Predicted Rating: 4.22
 9. Arts & Crafts                       | Predicted Rating: 4.21
10. Sports & Outdoors                   | Predicted Rating: 4.19

Customer ID: 29-392-9296
Top 10 Recommended Product Categories:
------------------------------------------------------------
 1. Baby Products                   

Saving SVD++ model predictions as a csv for fairness evaluation later:

In [22]:
# Initialize an empty list to hold each row as a dictionary
rows = []

# Iterate over the dictionary of predictions
for customer, recommendations in SVDpp_top_10_categories_per_customer.items():
    for rec in recommendations:
        # Each rec is a tuple (category, predicted_rating)
        rows.append({
            'customer_id': customer,
            'purchase_category': rec[0],
            'predicted_rating': rec[1]
        })

# Convert the list of rows into a DataFrame
svdpp_recommendations = pd.DataFrame(rows)

# Save the DataFrame to a CSV file
svdpp_recommendations.to_csv('svdpp_predictions_all.csv', index=False)

print("CSV file created successfully!")

CSV file created successfully!


---
---
Model 2: Item-based KNN (Hybrid model; Collaborative & Content-based)
---

A hybrid recommender system is a type of recommendation model that integrates two (or more) different recommendation strategies to leverage their complementary strengths. The goal is to provide more accurate, diverse, and fair recommendations by compensating for the limitations of individual models.

In this implementation, the hybrid model combines:

- Collaborative Filtering, which is based on the idea that users with similar rating behaviors will also like similar items.
- Content-Based Filtering, which builds a feature vector for each item (customer) and computes feature-based cosine similarity between them.

This enables the model to recommend items (customers) to users (purchase category) based on similar user characteristics, even if rating data is sparse.

---
The objective of this model is to build a recommendation system using an hybrid item-based KNN model that allows for personalize recommendations by considering both behavioral similarity (purchase patterns) and customer profiles (demographics, preferences, etc.). Similarly as with the SVD++ model:
- Product categories are treated as users
- Customers are treated as items
- Ratings are derived from custom scores, and predictions are based on a hybrid of rating similarity and user features

The process of the KNN model is as follows:
1. Flipping User-Item Matrix:
    - Creating a user-item matrix where:
        - Rows = product categories (Purchase_Category)
        - Columns = customers (Customer_ID)
        - Values = custom ratings (combination of product rating, satisfaction, loyalty)
        - Missing values filled with 0 for matrix computation.

2. Train-Test Split (Row-Wise)
    - For each product category (row), split its available ratings into: 80% training and 20% testing
    - This ensures fair category-level evaluation.

3. Customer Feature Engineering
    - Extracting customer-level features including demographics, behavioral traits, engagement, satisfaction, etc.
    - Applying one-hot encoding to categorical features.
    - Preparing these for use in feature-based similarity calculations.

4. Hybrid Similarity Matrix
    - Computing two types of cosine similarity between customers:
        - Rating-based: From customer rating patterns in the training matrix.
        - Feature-based: From encoded customer features.
    - Combining these into a hybrid similarity matrix:
        - Hybrid Similarity = 𝛼 ⋅ Rating Similarity + (1 − 𝛼) ⋅ Feature Similarity, where 𝛼 = 0.5 (equal weights).

5. Prediction Function
    - Defining predict_rating_hybrid():
        - For a given (product category, customer) pair,
        - Find the k-most similar customers,
        - Use their ratings (if available) to compute a weighted average prediction.

6. Model Evaluation
    - Evaluating the model using Root Mean Squared Error (RMSE) and Mean Absolute Error (MAE)

7. Recommendation Generation
    - Generating two top-N recommendation lists:
        - Top 10 customers per product category (which customers to target)
        - Top 10 product categories per customer (what customers might like most)

---
Importing necessary libraries:

In [23]:
from sklearn.metrics.pairwise import cosine_similarity
"""a similarity metric that measures the cosine of the angle between two vectors in a multi-dimensional space"""

from sklearn.metrics import mean_squared_error, mean_absolute_error
"""imports two commonly used evaluation metrics from Scikit-learn"""

'imports two commonly used evaluation metrics from Scikit-learn'

Creating the User-Item Matrix for recommendations, where the rating is the custom rating. This matrix becomes the base for train-test splitting and similarity calculations:

In [24]:
# Create the user-item matrix
# Rows: Purchase categories (as users), Columns: Customers (as items)
user_item_matrix = e_commerce.pivot_table(index='Purchase_Category', columns='Customer_ID', values='custom_rating')
user_item_matrix = user_item_matrix.fillna(0)

user_item_matrix.head()

Customer_ID,00-107-4749,00-149-4481,00-264-3797,00-265-0556,00-275-9990,00-285-9607,00-335-5034,00-477-9449,00-496-3067,00-520-4376,00-733-8770,01-023-2094,01-089-2223,01-404-3425,01-484-8220,01-799-2615,01-925-4021,02-004-0620,02-055-8248,02-100-4320,02-653-5884,02-676-9312,02-677-6735,02-711-1034,02-725-7664,02-911-0447,03-023-9932,03-102-0288,03-180-9200,03-422-8710,03-623-6767,03-669-7519,03-827-4021,03-874-9055,03-950-6074,03-977-8103,03-991-2098,03-992-3728,04-052-5592,04-213-4617,04-349-6759,04-352-1558,04-465-1819,04-468-5487,04-545-9162,04-563-9091,04-609-7885,04-660-2742,04-809-8989,04-829-7037,04-832-3559,05-067-9686,05-167-0833,05-465-8765,05-506-7216,05-546-5128,05-710-6109,05-857-1572,05-975-6985,06-008-2805,06-061-0023,06-160-8063,06-237-7338,06-357-6548,06-594-3417,06-638-0433,06-647-3248,06-764-5496,06-767-5206,07-024-4995,07-211-9364,07-314-2945,07-323-0255,07-395-5818,07-402-7433,07-479-3919,07-560-6637,07-751-9620,07-816-2673,07-834-3350,07-958-2231,07-966-8728,08-117-9893,08-185-6608,08-297-3351,08-584-8296,08-614-6177,08-652-2085,08-733-0300,08-755-1170,08-806-0588,08-977-4418,09-216-1957,09-258-0424,09-332-0236,09-346-4992,09-369-0588,09-443-7536,09-513-4923,09-737-8354,09-775-4058,09-783-7921,09-948-3150,10-079-8237,10-232-0685,10-247-0192,10-300-8072,10-328-5732,10-330-6387,10-430-5821,10-870-3357,10-961-9336,11-121-0709,11-719-8800,11-802-5604,11-820-5039,11-850-1953,11-975-9054,12-029-9344,12-409-5166,12-417-5609,12-440-5464,12-699-6543,12-839-8066,12-851-1771,13-015-8097,13-297-9777,13-571-6506,13-588-2488,13-588-5783,13-618-9438,13-792-4356,13-848-5757,14-025-2335,14-118-5229,14-249-1256,14-305-8712,14-364-4904,14-415-2802,14-603-9698,14-709-8547,14-784-5606,14-854-9545,15-146-4456,15-152-9751,15-354-9904,15-421-1255,15-530-3861,15-625-4047,15-640-6493,15-663-7994,15-711-4107,16-070-0138,16-364-8096,16-583-2801,16-730-2056,17-032-6274,17-198-5155,17-213-2834,17-246-6194,17-328-4678,17-344-8603,17-364-8685,17-551-9779,17-648-0171,17-763-9416,18-047-5540,18-097-5707,18-263-0366,18-452-1635,18-518-0140,18-593-8611,18-605-0795,18-614-2070,18-633-9387,18-655-3670,18-793-9543,18-832-6916,18-841-7253,18-925-1867,18-938-9896,19-116-4723,19-184-7665,19-311-9149,19-344-0102,19-383-9741,19-386-9088,19-560-0717,19-643-1879,19-667-0201,19-689-4556,19-873-9468,19-885-2597,19-925-2376,19-933-8095,19-999-0134,20-157-8631,20-419-0341,20-421-2544,20-460-9796,20-562-2569,20-648-1010,20-699-9217,20-817-9584,20-830-4758,20-939-3906,21-048-8669,21-232-1397,21-297-1223,21-300-2001,21-313-7424,21-734-0212,21-755-8745,21-783-9242,21-808-2293,21-821-6398,21-959-5766,22-022-9256,22-039-6819,22-053-4627,22-106-8347,22-157-2494,22-234-9315,22-311-5460,22-352-0576,22-382-9636,22-414-4955,22-483-3115,22-579-3978,22-692-3470,22-860-9860,22-981-6595,23-039-9347,23-066-1915,23-085-0811,23-246-5927,23-253-2565,23-302-7502,23-451-9920,23-537-0742,23-617-2637,23-752-2213,23-758-1012,23-794-8196,23-949-1954,23-985-3535,24-109-2173,24-241-7171,24-272-6205,24-371-5870,24-591-4327,24-662-2119,24-853-1635,24-878-4507,25-085-7917,25-096-6830,25-285-7453,25-286-8786,25-485-2023,25-502-1415,25-560-4506,25-839-8670,25-866-7286,26-028-2671,26-150-0209,26-235-6745,26-521-8328,26-596-5461,26-672-1285,26-828-2720,27-010-9462,27-017-7173,27-018-2262,27-100-0661,27-111-2242,27-262-6874,27-347-7437,27-439-9545,27-463-5671,27-543-7069,27-571-9402,27-613-5005,27-619-9264,27-672-6081,27-742-2776,27-806-4972,27-843-5553,27-941-4995,28-123-8406,28-225-4286,28-395-9872,28-676-0614,28-764-3337,28-832-2885,28-855-1639,28-862-5550,28-874-8938,28-947-4295,28-969-7540,29-067-4901,29-319-4519,29-341-6463,29-374-1297,29-392-9296,29-438-8126,29-625-7057,29-779-3457,29-779-7209,29-827-8515,29-987-3969,30-038-8752,30-083-9565,30-133-5120,30-417-6981,30-520-8656,30-522-1574,30-528-9105,30-567-4471,30-735-8744,30-839-5589,30-975-0306,30-987-7421,31-060-2716,31-088-6950,31-147-6197,31-218-0024,31-284-5722,31-364-4144,31-395-1286,31-491-8457,31-783-9785,31-788-0864,31-9

Performing train-test split by splitting ratings within each category (row) into 80% train and 20% test. This split ensures all categories get evaluated and not just globally split:

In [25]:
# Split the user-item matrix into train and test matrices row-wise (80/20)
train_matrix = user_item_matrix.copy() * np.nan
test_matrix = user_item_matrix.copy() * np.nan
results = []

for category in user_item_matrix.index:
    ratings = user_item_matrix.loc[category]
    non_zero_ratings = ratings[ratings > 0]
    shuffled_ratings = non_zero_ratings.sample(frac=1, random_state=42)
    cutoff = int(0.8 * len(shuffled_ratings))
    
    train_matrix.loc[category, shuffled_ratings.index[:cutoff]] = shuffled_ratings.values[:cutoff]
    test_matrix.loc[category, shuffled_ratings.index[cutoff:]] = shuffled_ratings.values[cutoff:]
    
    results.append({
        'userID': category,
        'Total_Ratings': len(non_zero_ratings),
        'Train_Ratings': cutoff,
        'Test_Ratings': len(non_zero_ratings) - cutoff,
        'Train_%': (cutoff / len(non_zero_ratings)) * 100,
        'Test_%': ((len(non_zero_ratings) - cutoff) / len(non_zero_ratings)) * 100
    })

train_test_results_df = pd.DataFrame(results)
display(train_test_results_df)

,userID,Total_Ratings,Train_Ratings,Test_Ratings,Train_%,Test_%
0,Animal Feed,44,35,9,79.545455,20.454545
1,Arts & Crafts,34,27,7,79.411765,20.588235
2,Baby Products,41,32,9,78.048780,21.951220
3,Beauty & Personal Care,34,27,7,79.411765,20.588235
4,Books,37,29,8,78.378378,21.621622
5,Clothing,30,24,6,80.000000,20.000000
6,Electronics,54,43,11,79.629630,20.370370
7,Food & Beverages,44,35,9,79.545455,20.454545
8,Furniture,41,32,9,78.048780,21.951220
9,Gardening & Outdoors,44,35,9,79.545455,20.454545


Preparing customer feature vectors:

In [26]:
# Prepare customer feature DataFrame for similarity computation

# Define the list of features to be used
features = [
    'Customer_ID', 'Age', 'Gender', 'Income_Level', 'Marital_Status', 'Education_Level',
    'Occupation', 'Purchase_Amount', 'Frequency_of_Purchase', 'Purchase_Channel',
    'Brand_Loyalty', 'Product_Rating', 'Time_Spent_on_Product_Research(hours)',
    'Social_Media_Influence', 'Discount_Sensitivity', 'Customer_Satisfaction',
    'Engagement_with_Ads', 'Device_Used_for_Shopping', 'Payment_Method',
    'Time_to_Decision', 'Customer_Loyalty_Program_Member', 'Purchase_Intent',
    'Shipping_Preference'
]

# Filter and copy the relevant columns
customer_features_df = e_commerce[features].copy()

# Convert boolean-like column to integer
customer_features_df['Customer_Loyalty_Program_Member'] = customer_features_df['Customer_Loyalty_Program_Member'].astype(int)

# One-hot encode categorical variables to prepare them for cosine similarity computation
categorical_cols = [
    'Gender', 'Income_Level', 'Marital_Status', 'Education_Level',
    'Occupation', 'Purchase_Channel', 'Social_Media_Influence',
    'Discount_Sensitivity', 'Engagement_with_Ads', 'Device_Used_for_Shopping',
    'Payment_Method', 'Purchase_Intent', 'Shipping_Preference'
]
customer_features_encoded = pd.get_dummies(customer_features_df, columns=categorical_cols, drop_first=True)

# Define numeric columns (left unnormalized for cosine similarity)
numeric_cols = [
    'Age', 'Purchase_Amount', 'Frequency_of_Purchase', 'Brand_Loyalty',
    'Product_Rating', 'Time_Spent_on_Product_Research(hours)', 'Customer_Satisfaction',
    'Time_to_Decision'
]

The code above prepares customer-level features and applies one-hot encoding to categorical columns. These features will be used for feature-based similarity:

In [27]:
"""
This block computes two types of cosine similarities between customers:

1. **Rating-based similarity**: 
   - Based on customers’ rating behavior in the training matrix.
   - Cosine similarity is computed between customer rating vectors (columns of train_matrix).

2. **Feature-based similarity**:
   - Based on encoded customer demographic and behavioral features.
   - Cosine similarity is computed from one-hot encoded and numerical features.

These two similarity matrices are then combined into a hybrid similarity matrix
using a weighted average controlled by the parameter `alpha` (here set to 0.5, 
implying equal contribution from both rating and feature-based similarities).
"""

# Step 1: Rating-based similarity
train_matrix_filled = train_matrix.fillna(0)
rating_similarity_df = pd.DataFrame(
    cosine_similarity(train_matrix_filled.T),
    index=train_matrix.columns,
    columns=train_matrix.columns
)

# Step 2: Feature-based similarity
if 'Customer_ID' in customer_features_encoded.columns:
    customer_features_encoded = customer_features_encoded.set_index('Customer_ID')

feature_similarity_df = pd.DataFrame(
    cosine_similarity(customer_features_encoded),
    index=customer_features_encoded.index,
    columns=customer_features_encoded.index
)

# Step 3: Combine the similarities
alpha = 0.5
hybrid_similarity_df = alpha * rating_similarity_df + (1 - alpha) * feature_similarity_df

(hybrid_similarity_df.head())

Customer_ID,00-107-4749,00-149-4481,00-264-3797,00-265-0556,00-275-9990,00-285-9607,00-335-5034,00-477-9449,00-496-3067,00-520-4376,00-733-8770,01-023-2094,01-089-2223,01-404-3425,01-484-8220,01-799-2615,01-925-4021,02-004-0620,02-055-8248,02-100-4320,02-653-5884,02-676-9312,02-677-6735,02-711-1034,02-725-7664,02-911-0447,03-023-9932,03-102-0288,03-180-9200,03-422-8710,03-623-6767,03-669-7519,03-827-4021,03-874-9055,03-950-6074,03-977-8103,03-991-2098,03-992-3728,04-052-5592,04-213-4617,04-349-6759,04-352-1558,04-465-1819,04-468-5487,04-545-9162,04-563-9091,04-609-7885,04-660-2742,04-809-8989,04-829-7037,04-832-3559,05-067-9686,05-167-0833,05-465-8765,05-506-7216,05-546-5128,05-710-6109,05-857-1572,05-975-6985,06-008-2805,06-061-0023,06-160-8063,06-237-7338,06-357-6548,06-594-3417,06-638-0433,06-647-3248,06-764-5496,06-767-5206,07-024-4995,07-211-9364,07-314-2945,07-323-0255,07-395-5818,07-402-7433,07-479-3919,07-560-6637,07-751-9620,07-816-2673,07-834-3350,07-958-2231,07-966-8728,08-117-9893,08-185-6608,08-297-3351,08-584-8296,08-614-6177,08-652-2085,08-733-0300,08-755-1170,08-806-0588,08-977-4418,09-216-1957,09-258-0424,09-332-0236,09-346-4992,09-369-0588,09-443-7536,09-513-4923,09-737-8354,09-775-4058,09-783-7921,09-948-3150,10-079-8237,10-232-0685,10-247-0192,10-300-8072,10-328-5732,10-330-6387,10-430-5821,10-870-3357,10-961-9336,11-121-0709,11-719-8800,11-802-5604,11-820-5039,11-850-1953,11-975-9054,12-029-9344,12-409-5166,12-417-5609,12-440-5464,12-699-6543,12-839-8066,12-851-1771,13-015-8097,13-297-9777,13-571-6506,13-588-2488,13-588-5783,13-618-9438,13-792-4356,13-848-5757,14-025-2335,14-118-5229,14-249-1256,14-305-8712,14-364-4904,14-415-2802,14-603-9698,14-709-8547,14-784-5606,14-854-9545,15-146-4456,15-152-9751,15-354-9904,15-421-1255,15-530-3861,15-625-4047,15-640-6493,15-663-7994,15-711-4107,16-070-0138,16-364-8096,16-583-2801,16-730-2056,17-032-6274,17-198-5155,17-213-2834,17-246-6194,17-328-4678,17-344-8603,17-364-8685,17-551-9779,17-648-0171,17-763-9416,18-047-5540,18-097-5707,18-263-0366,18-452-1635,18-518-0140,18-593-8611,18-605-0795,18-614-2070,18-633-9387,18-655-3670,18-793-9543,18-832-6916,18-841-7253,18-925-1867,18-938-9896,19-116-4723,19-184-7665,19-311-9149,19-344-0102,19-383-9741,19-386-9088,19-560-0717,19-643-1879,19-667-0201,19-689-4556,19-873-9468,19-885-2597,19-925-2376,19-933-8095,19-999-0134,20-157-8631,20-419-0341,20-421-2544,20-460-9796,20-562-2569,20-648-1010,20-699-9217,20-817-9584,20-830-4758,20-939-3906,21-048-8669,21-232-1397,21-297-1223,21-300-2001,21-313-7424,21-734-0212,21-755-8745,21-783-9242,21-808-2293,21-821-6398,21-959-5766,22-022-9256,22-039-6819,22-053-4627,22-106-8347,22-157-2494,22-234-9315,22-311-5460,22-352-0576,22-382-9636,22-414-4955,22-483-3115,22-579-3978,22-692-3470,22-860-9860,22-981-6595,23-039-9347,23-066-1915,23-085-0811,23-246-5927,23-253-2565,23-302-7502,23-451-9920,23-537-0742,23-617-2637,23-752-2213,23-758-1012,23-794-8196,23-949-1954,23-985-3535,24-109-2173,24-241-7171,24-272-6205,24-371-5870,24-591-4327,24-662-2119,24-853-1635,24-878-4507,25-085-7917,25-096-6830,25-285-7453,25-286-8786,25-485-2023,25-502-1415,25-560-4506,25-839-8670,25-866-7286,26-028-2671,26-150-0209,26-235-6745,26-521-8328,26-596-5461,26-672-1285,26-828-2720,27-010-9462,27-017-7173,27-018-2262,27-100-0661,27-111-2242,27-262-6874,27-347-7437,27-439-9545,27-463-5671,27-543-7069,27-571-9402,27-613-5005,27-619-9264,27-672-6081,27-742-2776,27-806-4972,27-843-5553,27-941-4995,28-123-8406,28-225-4286,28-395-9872,28-676-0614,28-764-3337,28-832-2885,28-855-1639,28-862-5550,28-874-8938,28-947-4295,28-969-7540,29-067-4901,29-319-4519,29-341-6463,29-374-1297,29-392-9296,29-438-8126,29-625-7057,29-779-3457,29-779-7209,29-827-8515,29-987-3969,30-038-8752,30-083-9565,30-133-5120,30-417-6981,30-520-8656,30-522-1574,30-528-9105,30-567-4471,30-735-8744,30-839-5589,30-975-0306,30-987-7421,31-060-2716,31-088-6950,31-147-6197,31-218-0024,31-284-5722,31-364-4144,31-395-1286,31-491-8457,31-783-9785,31-788-0864,31-9

Hybrid KNN prediction function:

In [28]:
def predict_rating_hybrid(user_id, item_id, k=5):
    """
    Predicts a rating for a given (category, customer) pair using KNN
    with a hybrid similarity of rating and feature vectors.
    """
    similar_items = hybrid_similarity_df[item_id].sort_values(ascending=False).iloc[1:k+1]
    category_ratings = train_matrix.loc[user_id, similar_items.index].dropna()
    
    if category_ratings.empty:
        return np.nan
    
    weighted_sum = (similar_items.loc[category_ratings.index] * category_ratings).sum()
    total_weight = similar_items.loc[category_ratings.index].sum()
    
    return weighted_sum / total_weight if total_weight else np.nan

Evaluating the KNN model performance:

In [29]:
test_data = test_matrix.stack().reset_index(name='true_rating')
test_data.columns = ['user_id', 'item_id', 'true_rating']

predictions, actuals = [], []
for row in test_data.itertuples(index=False):
    pred = predict_rating_hybrid(row.user_id, row.item_id, k=5)
    if not np.isnan(pred):
        predictions.append(pred)
        actuals.append(row.true_rating)

if predictions:
    rmse = np.sqrt(mean_squared_error(actuals, predictions))
    mae = mean_absolute_error(actuals, predictions)
    print(f"RMSE: {rmse:.4f}")
    print(f"MAE: {mae:.4f}")

RMSE: 0.9094
MAE: 0.7163


For each product category, predicting ratings for all customers and keeping the top 10 customers with the highest predicted scores:

In [30]:
"""
This block predicts the top 10 customers for each product category 
using the hybrid ItemKNN model. For demonstration purposes, 
only the results for 3 product categories are printed.
"""

# Step 1: Predict ratings for each (category, customer) pair
top_10_customers_per_category = defaultdict(list)

for category in categories:
    for customer in customers:
        pred = predict_rating_hybrid(user_id=category, item_id=customer)
        if not np.isnan(pred):
            top_10_customers_per_category[category].append((customer, pred))

# Step 2: Sort predictions and keep top 10 customers per category
for category in top_10_customers_per_category:
    top_10_customers_per_category[category] = sorted(
        top_10_customers_per_category[category],
        key=lambda x: x[1],
        reverse=True
    )[:10]

# Step 3: Display results for only 3 sample product categories
print("Top 10 recommended customers for 3 sample product categories:\n")

for i, (category, top_customers) in enumerate(top_10_customers_per_category.items()):
    if i >= 3:
        break
    print(f"Product Category: {category}")
    for idx, (cust, pred_rating) in enumerate(top_customers, 1):
        print(f"{idx}. Customer ID: {cust} — Predicted Rating: {pred_rating:.2f}")
    print("=" * 60)

Top 10 recommended customers for 3 sample product categories:

Product Category: Gardening & Outdoors
1. Customer ID: 83-714-9649 — Predicted Rating: 6.30
2. Customer ID: 37-611-6911 — Predicted Rating: 5.20
3. Customer ID: 84-459-7736 — Predicted Rating: 5.20
4. Customer ID: 60-470-3563 — Predicted Rating: 5.20
5. Customer ID: 52-726-9328 — Predicted Rating: 5.00
6. Customer ID: 29-374-1297 — Predicted Rating: 5.00
7. Customer ID: 67-697-2331 — Predicted Rating: 5.00
8. Customer ID: 27-017-7173 — Predicted Rating: 3.90
9. Customer ID: 86-257-9581 — Predicted Rating: 3.86
10. Customer ID: 30-528-9105 — Predicted Rating: 3.78
Product Category: Food & Beverages
1. Customer ID: 27-347-7437 — Predicted Rating: 5.40
2. Customer ID: 84-894-9222 — Predicted Rating: 5.20
3. Customer ID: 71-876-9118 — Predicted Rating: 5.20
4. Customer ID: 35-850-8676 — Predicted Rating: 5.20
5. Customer ID: 61-926-1691 — Predicted Rating: 4.80
6. Customer ID: 22-352-0576 — Predicted Rating: 4.60
7. Customer ID

Flipping the logic back and finding for each customer the top 10 product categories they are most likely to rate highly based on hybrid KNN predictions:

In [31]:
# Step 1: Predict ratings for each (customer, category) pair
knn_top_recommendations_per_customer = defaultdict(list)

for customer in customers:
    for category in categories:
        pred = predict_rating_hybrid(user_id=category, item_id=customer)
        if not np.isnan(pred):
            knn_top_recommendations_per_customer[customer].append((category, pred))

# Step 2: Sort and keep top 10 categories per customer
for customer in knn_top_recommendations_per_customer:
    knn_top_recommendations_per_customer[customer] = sorted(
        knn_top_recommendations_per_customer[customer],
        key=lambda x: x[1],
        reverse=True
    )[:10]

# Step 3: Display recommendations for 5 sample customers
print("Top 10 recommended product categories for 5 sample customers:\n")
knn_top_categories_data = []

for i, (customer, top_categories) in enumerate(knn_top_recommendations_per_customer.items()):
    if i >= 5:
        break

    print("=" * 60)
    print(f"Customer ID: {customer}")
    print("Top Recommended Product Categories:")
    print("-" * 60)
    
    for idx, (category, pred_rating) in enumerate(top_categories, 1):
        knn_top_categories_data.append({
            "Customer ID": customer,
            "Rank": idx,
            "Product Category": category,
            "Predicted Rating": round(pred_rating, 2)
        })
        print(f"{idx:>2}. {category:<35} | Predicted Rating: {pred_rating:.2f}")
    
    print("=" * 60 + "\n")

Top 10 recommended product categories for 5 sample customers:

Customer ID: 37-611-6911
Top Recommended Product Categories:
------------------------------------------------------------
 1. Office Supplies                     | Predicted Rating: 5.60
 2. Gardening & Outdoors                | Predicted Rating: 5.20
 3. Software & Apps                     | Predicted Rating: 4.70
 4. Baby Products                       | Predicted Rating: 4.60
 5. Food & Beverages                    | Predicted Rating: 4.30

Customer ID: 29-392-9296
Top Recommended Product Categories:
------------------------------------------------------------
 1. Food & Beverages                    | Predicted Rating: 3.00

Customer ID: 84-649-5117
Top Recommended Product Categories:
------------------------------------------------------------
 1. Office Supplies                     | Predicted Rating: 3.44

Customer ID: 48-980-6078
Top Recommended Product Categories:
----------------------------------------------------

Because the data is so sparse, for some customers the model can't predict for all the customers even top 10 product categories.

Saving all the predictions from ItemKNN for further analysis:

In [32]:
# List to store all predictions
itemknn_prediction_data = []

# Iterate over every purchase category (user_id) and customer (item_id)
for purchase_category in user_item_matrix.index:
    for customer_id in user_item_matrix.columns:
        pred = predict_rating_hybrid(purchase_category, customer_id, k=5)
        
        # Optionally, you can retrieve the "true" rating if you want.
        # Because user_item_matrix has 0 where there's no rating,
        # we'll store it but keep in mind that 0 may indicate a missing rating.
        true_rating = user_item_matrix.loc[purchase_category, customer_id]
        
        itemknn_prediction_data.append({
            'purchase_category': purchase_category,
            'customer_id': customer_id,
            'true_rating': true_rating,
            'predicted_rating': pred
        })

# Convert the list of predictions to a DataFrame
itemknn_predictions_df = pd.DataFrame(itemknn_prediction_data)

# Save the DataFrame to a CSV file
itemknn_predictions_df.to_csv('itemknn_predictions_all.csv', index=False)

print("CSV file with all KNN predictions saved successfully!")

CSV file with all KNN predictions saved successfully!


---